In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [2]:
data=pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
test_data=pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")

In [3]:
X=data.drop(columns='Fertilizer Name',axis=1)
y=data['Fertilizer Name']
X_test=test_data.copy()

X_combined=pd.concat([X,X_test],axis=0)
X_combined=pd.get_dummies(X_combined).astype(int)

X=X_combined.iloc[:len(X), :]
X_test=X_combined.iloc[len(X):, :]

In [4]:
from sklearn.preprocessing import LabelEncoder

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y_encoded)

In [6]:
from xgboost import XGBClassifier

model=XGBClassifier()
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [7]:
def make_predictions(inputs, model):
    probs = model.predict_proba(inputs)
    top3_idx = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
    predictions = le_target.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)
    return predictions

In [8]:
val_predictions = make_predictions(X_val, model)


In [9]:
val_predictions


array([['10-26-26', '14-35-14', '17-17-17'],
       ['10-26-26', '14-35-14', '20-20'],
       ['17-17-17', 'DAP', '14-35-14'],
       ...,
       ['28-28', '20-20', '10-26-26'],
       ['28-28', 'Urea', '10-26-26'],
       ['14-35-14', '10-26-26', '17-17-17']], dtype=object)

In [10]:
test_predictions = make_predictions(X_test, model)
outputs = [' '.join(row) for row in test_predictions]

submission = pd.DataFrame({
    "id": test_data["id"],
    "Fertilizer Name": outputs
})
submission.to_csv("submission.csv", index=False)